# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is the SysML v2 Validation case 2a:

    package '2a-Parts Interconnection' {
	import Definitions::*;
	import Usages::*;

	package Definitions {		
		// Port Definitions
		
		port def FuelCmdPort;
		
		port def DrivePwrPort;
		port def ClutchPort;
		
		port def ShaftPort_a;
		port def ShaftPort_b;
		port def ShaftPort_c;
		port def ShaftPort_d;
		
		port def DiffPort;
		port def AxlePort;
		port def AxleToWheelPort;
		port def WheelToAxlePort;
		port def WheelToRoadPort;
		
		/**
		 * A port definition can have nested ports.
		 */
		port def VehicleToRoadPort { 
			port wheelToRoadPort: WheelToRoadPort[2];
		}
	
		// Blocks
	
		part def VehicleA { 
			port fuelCmdPort: FuelCmdPort;
			port vehicleToRoadPort: VehicleToRoadPort;
		}
		
		part def AxleAssembly;		
		part def RearAxleAssembly :> AxleAssembly { 
			port shaftPort_d: ShaftPort_d;
		}
		
		part def Axle;
		part def RearAxle :> Axle;
		
		part def HalfAxle { 
			port axleToDiffPort: AxlePort;
			port axleToWheelPort: AxleToWheelPort;
		}
		
		part def Engine { 
			port fuelCmdPort: FuelCmdPort;
			port drivePwrPort: DrivePwrPort;
		}
	
		part def Transmission { 
			port clutchPort: ClutchPort;
			port shaftPort_a: ShaftPort_a;
		}
		
		part def Driveshaft { 
			port shaftPort_b: ShaftPort_b;
			port shaftPort_c: ShaftPort_c;
		}	
		
		/**
		 * Ports do not have to be defined on part defs.
		 * They can be added directly to their usages.
		 */
		part def Differential;
		part def Wheel;
		
		// Interface Definitions
		
		/**
		 * The ends of an interface definition are always ports.
		 */
		interface def EngineToTransmissionInterface {
			end drivePwrPort: DrivePwrPort;
			end clutchPort: ClutchPort;
		}
		
		interface def DriveshaftInterface {
			end shaftPort_a: ShaftPort_a;
			end shaftPort_d: ShaftPort_d;
			
			/**
			 * 'driveshaft' is a reference to the driveshaft that will
			 * act as the "interface medium" for this interface.
			 */
			ref driveshaft: Driveshaft {
				port shaftPort_b :>> Driveshaft::shaftPort_b;
				port shaftPort_c :>> Driveshaft::shaftPort_c;
			}
			
			/**
			 * The two ends of 'DriveShaftInterface' are always connected
			 * via the referenced 'driveshaft'.
			 */
			connect shaftPort_a to driveshaft::shaftPort_b;
			connect driveshaft::shaftPort_c to shaftPort_d;
		}
		
	}
	
	package Usages {
	
		part vehicle1_c1: VehicleA {
			
			/**
			 * Inherited ports are redefined to provide "local connection points".
			 * ("::>>" is a shorthand for "redefines".)
			 */
			port fuelCmdPort :>> VehicleA::fuelCmdPort;
			
			bind fuelCmdPort = engine::fuelCmdPort;
			
			part engine: Engine {
				port fuelCmdPort :>> Engine::fuelCmdPort;
				port drivePwrPort :>> Engine::drivePwrPort;
			}
			
			/**
			 * A usage of an interface definition connects two ports relative to 
			 * a containing context.
			 */
			interface :EngineToTransmissionInterface
				connect engine::drivePwrPort to transmission::clutchPort;
				
			part transmission: Transmission {
				port clutchPort :>> Transmission::clutchPort;
				port shaftPort_a :>> Transmission::shaftPort_a;	
			}
			
			/**
			 * This 'driveshaft' is the part of 'vehicle1_c1' that will act as the
			 * interface medium in the following 'DriveshaftInterface' usage.
			 */
			part driveshaft: Driveshaft;
			
			interface :DriveshaftInterface
				connect transmission::shaftPort_a to rearAxleAssembly::shaftPort_d {
					/**
					 * The reference property from 'DriveshaftInterface' is redefined
					 * in order to bind it to the appropriate part of 'vehicle1_c1'.
					 */
					ref :>> DriveshaftInterface::driveshaft = vehicle1_c1::driveshaft;
				}
	
			part rearAxleAssembly: RearAxleAssembly {
				port shaftPort_d: ShaftPort_d :>> RearAxleAssembly::shaftPort_d;
	
				bind shaftPort_d = differential::shaftPort_d;
				
				part differential: Differential {
					/**
					 * If the part def has no ports, then they can be defined directly in
					 * a usage of the part def.
					 */
					port shaftPort_d: ShaftPort_d;
					port leftDiffPort: DiffPort;
					port rightDiffPort: DiffPort;
				}
				
				/* TODO: Allow use of "interface" keyword for an unnamed, default-typed interface usage. */
				
				/**
				 * A connection can be to a port that is arbitrarily deeply nested, on either end. 
				 */
				connect differential::leftDiffPort to rearAxle::leftHalfAxle::leftAxleToDiffPort;
				connect differential::rightDiffPort to rearAxle::rightHalfAxle::rightAxleToDiffPort;
		
				part rearAxle: RearAxle {
					part leftHalfAxle: HalfAxle {
						port leftAxleToDiffPort :>> HalfAxle::axleToDiffPort;
						port leftAxleToWheelPort :>> HalfAxle::axleToWheelPort;
					}
					part rightHalfAxle: HalfAxle  {
						port rightAxleToDiffPort :>> HalfAxle::axleToDiffPort;
						port rightAxleToWheelPort :>> HalfAxle::axleToWheelPort;
					}
				}
				
				connect rearAxle::leftHalfAxle::leftAxleToWheelPort to leftWheel::wheelToAxlePort;
				connect rearAxle::rightHalfAxle::rightAxleToWheelPort to rightWheel::wheelToAxlePort;
	
				part rearWheel: Wheel[2] ordered;
				
				/* The two rear wheels of 'rearAxleAssembly' must be given
				 * their own names in order to be referenced in connections.
				 * 
				 * (":>" is a shorthand here for "subsets".)
				 */
				part leftWheel :> rearWheel = rearWheel[1] {
					port wheelToAxlePort: WheelToAxlePort;
					port wheelToRoadPort: WheelToRoadPort;
				}
				
				part rightWheel :> rearWheel = rearWheel[2] {
					port wheelToAxlePort: WheelToAxlePort;
					port wheelToRoadPort: WheelToRoadPort;
				}
				
			}
			
			bind rearAxleAssembly::leftWheel::wheelToRoadPort = 
				 vehicleToRoadPort::leftWheelToRoadPort;
				 
			bind rearAxleAssembly::rightWheel::wheelToRoadPort = 
				 vehicleToRoadPort::rightWheelToRoadPort;
				
			port vehicleToRoadPort redefines VehicleA::vehicleToRoadPort {
				port leftWheelToRoadPort :> wheelToRoadPort = wheelToRoadPort[1];
				port rightWheelToRoadPort :> wheelToRoadPort = wheelToRoadPort[2];
			}
			
		}
	
	}
	
}

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import pymbe.api as pm
from pymbe.label import get_label
import networkx as nx
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.client import SysML2Client
from pymbe.query.metamodel_navigator import map_inputs_to_results

from pymbe.interpretation.calc_dependencies import generate_execution_order

import os

from pymbe.graph.lpg import SysML2LabeledPropertyGraph

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [2]:
helper_client = SysML2Client()

path = "C:\\Users\\bjorn\\Documents\\Git\\pyMBE"
file_name = "\\tests\\data\\Validation Case 2a\\elements.json"

helper_client._load_disk_elements(path + str(file_name))

Create a graph representation of the model and load it into memory.

In [3]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [4]:
shorten_pre_bake = {
}

In [5]:
helper_client.elements_by_id
empty_node_ids = [node_id for node_id in helper_client.elements_by_id if helper_client.elements_by_id[node_id] == {}]
empty_node_ids

[]

In [6]:
empty_node_ids = [node_id for node_id in lpg.nodes if lpg.nodes[node_id] == {}]
empty_node_ids

[]

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [7]:
m0_interpretation = random_generator_playbook(
    helper_client,
    lpg,
    shorten_pre_bake
)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:284: UserWarning: These edge types are not in the graph: {'ResultExpressionMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


NotImplementedError: Cannot handle untyped features!

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [8]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

['Driveshaft', 'shaftPort_b: ShaftPort_b'], 1
['Driveshaft', 'shaftPort_c: ShaftPort_c'], 1
['VehicleToRoadPort', 'wheelToRoadPort: WheelToRoadPort'], 2
['vehicle1_c1: VehicleA', 'rearAxleAssembly: RearAxleAssembly', 'rearAxle: RearAxle', 'rightHalfAxle: HalfAxle', 'rightAxleToDiffPort: AxlePort'], 1
['vehicle1_c1: VehicleA', 'rearAxleAssembly: RearAxleAssembly', 'rearAxle: RearAxle', 'rightHalfAxle: HalfAxle', 'rightAxleToWheelPort: AxleToWheelPort'], 1
['vehicle1_c1: VehicleA', 'rearAxleAssembly: RearAxleAssembly', 'leftWheel: Wheel', 'wheelToAxlePort: WheelToAxlePort'], 2
['vehicle1_c1: VehicleA', 'rearAxleAssembly: RearAxleAssembly', 'leftWheel: Wheel', 'wheelToRoadPort: WheelToRoadPort'], 2
['vehicle1_c1: VehicleA', 'fuelCmdPort: FuelCmdPort'], 1
['vehicle1_c1: VehicleA', 'driveshaft: Driveshaft'], 1
['vehicle1_c1: VehicleA', 'engine: Engine', 'drivePwrPort: DrivePwrPort'], 1
['vehicle1_c1: VehicleA', 'engine: Engine', 'fuelCmdPort: FuelCmdPort'], 1
['vehicle1_c1: VehicleA', 'rear

Once the interpretations are generated, we can look for expressions and create an execution order (this is similar to Excel builds a dependency graph internally to accelerate computations and partial updates when a user changes a cell value).

In [8]:
dcg = generate_execution_order(lpg, m0_interpretation)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:267: UserWarning: These edge types are not in the graph: {'ImpliedParameterFeedforward'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


One of the core tools in examining and working with the M1 model is using the get_projection function on the master graph to select out the kind of nodes and edges that will support other queries (roll-up using breadth-first search in reverse order, paths from one node to another to lay out sequences, etc.)

In [9]:
from pymbe.graph.calc_lpg import CalculationGroup
cg = CalculationGroup(lpg.get_projection("Expression Inferred Graph"), m0_interpretation, dcg)

Display the calculation order determined by the algorithm, as well as a hint about what the step is (generating output from a function, moving values from function parameters to an attribute, applying redefinition, etc.)

In [10]:
for item in dcg:
    instance_safe = True
    if item[0] not in m0_interpretation or len(m0_interpretation[item[0]]) == 0:
        print("No instances for " + lpg.nodes[item[0]]['qualifiedName'])
        instance_safe = False
    if item[1] not in m0_interpretation or len(m0_interpretation[item[1]]) == 0:
        print("No instances for " + lpg.nodes[item[1]]['qualifiedName'])
        instance_safe = False
    if instance_safe:
        rep_source = m0_interpretation[item[0]][0][-1]
        rep_target = m0_interpretation[item[1]][0][-1]
        if len(item) == 3:
            safe_item = item[2]
        else:
            safe_item = 'None'
        print('(' + str(rep_source) + ', ' + str(rep_target) + ', ' + safe_item + ')')

(1 «Occurred LiteralInteger», Rocket#0.LS#3.1 «Occurred LiteralInteger».$result#0 (unset), Output)
(162.91 «Occurred LiteralReal», RT-5#0.162.91 «Occurred LiteralReal».$result#0 (unset), Output)
(197.9 «Occurred LiteralReal», RT-10#0.197.9 «Occurred LiteralReal».$result#0 (unset), Output)
(RT-5#0.162.91 «Occurred LiteralReal».$result#0 (unset), .Real#1317 (unset), ValueBinding)
(RT-10#0.197.9 «Occurred LiteralReal».$result#0 (unset), .Real#1292 (unset), ValueBinding)
(.Real#1317 (unset), .Real#127 (unset), Redefinition)
(.Real#1292 (unset), .Real#127 (unset), Redefinition)
(0.45 «Occurred LiteralReal», RT-5#0.0.45 «Occurred LiteralReal».$result#0 (unset), Output)
(0.75 «Occurred LiteralReal», RT-10#0.0.75 «Occurred LiteralReal».$result#0 (unset), Output)
(RT-5#0.0.45 «Occurred LiteralReal».$result#0 (unset), .Real#2669 (unset), ValueBinding)
(RT-10#0.0.75 «Occurred LiteralReal».$result#0 (unset), .Real#2300 (unset), ValueBinding)
(.Real#2669 (unset), .Real#2518 (unset), Redefinition)
(

Use the calculation order in order to resolve the "unset" fields on many attributes to values where they are determined in the M1 model (e.g., using the ' = ' operator to assign values directly or connect to equations or analyses).

In [11]:
cg.solve_graph(lpg)

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

In [12]:
m0_interpretation[collect_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#0 ([.Real#247 (0.5625), .Real#2601 (0.5625), .Real#2803 (0.5625), .Real#1855 (0.5625), .Real#198 (0.5625), .Real#1938 (0.5625), .Real#461 (0.5625), .Real#1169 (0.5625), .Real#76 (1.125), .Real#524 (0.5625), .Real#2453 (0.5625), .Real#2871 (0.5625), .Real#2581 (0.5625), .Real#799 (0.5625), .Real#1344 (0.5625), .Real#1555 (0.5625), .Real#1936 (0.5625), .Real#2148 (0.5625), .Real#2067 (0.5625), .Real#2958 (0.5625), .Real#980 (0.5625), .Real#633 (0.5625), .Real#2348 (0.5625), .Real#337 (0.5625), .Real#62 (0.5625), .Real#874 (0.5625), .Real#1814 (0.5625), .Real#2227 (0.5625), .Real#2680 (1.125), .Real#1603 (0.5625)])],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collectio

In [13]:
m0_interpretation[collect_2_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#0 ([.Real#1368 (1.5), .Real#2495 (1.5), .Real#860 (1.5), .Real#1735 (1.5), .Real#1354 (1.5)])],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#1 ([])],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#2 ([.Real#2296 (1.5), .Real#1096 (1.5), .Real#2590 (1.5), .Real#1294 (1.5)])],
 [LS#3,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  LS#3.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.$result#3 ([.Real#805 (1

In [14]:
m0_interpretation[sum_2_collection]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$collection#0 ([.Real#1368 (1.5), .Real#2495 (1.5), .Real#860 (1.5), .Real#1735 (1.5), .Real#1354 (1.5)])],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.$collection#1 ([])],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.$collection#2 ([.Real#2296 (1.5), .Real#1096 (1.5), .Real#2590 (1.5), .Real#1294 (1.5)])],
 [LS#3,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#3.+ ($x, $y) => $result.sum ($collection) => $result.$collection#3 ([.Real#805 (1.5), .Real#2923 (1.5), .Real#1723 (1.5), .Real#392 (1.5)])]]

In [15]:
m0_interpretation[sum_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.$result#0 (18.0)],
 [LS#1,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#1.+ ($x, $y) => $result.sum ($collection) => $result.$result#1 (1.125)],
 [LS#2,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#2.+ ($x, $y) => $result.sum ($collection) => $result.$result#2 (4.5)],
 [LS#3,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  LS#3.+ ($x, $y) => $result.sum ($collection) => $result.$result#3 (15.1875)]]

In [16]:
m0_interpretation[x]

[[LS#0, + ($x, $y) => $result, LS#0.+ ($x, $y) => $result.$x#0 (7.5)],
 [LS#1, + ($x, $y) => $result, LS#1.+ ($x, $y) => $result.$x#1 (0)],
 [LS#2, + ($x, $y) => $result, LS#2.+ ($x, $y) => $result.$x#2 (6.0)],
 [LS#3, + ($x, $y) => $result, LS#3.+ ($x, $y) => $result.$x#3 (6.0)]]

In [17]:
m0_interpretation[liquid_stage_full_mass]

[[LS#0, .Real#253 (25.5)],
 [LS#1, .Real#2997 (1.125)],
 [LS#2, .Real#616 (10.5)],
 [LS#3, .Real#485 (21.1875)]]

In [18]:
m0_interpretation[fre_1_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Full Mass (p) => $result,
  FRE.Full Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Full Mass (p) => $result.FRE.Full Mass.$result#0 ([[FL-T200#0, .Real#765 (1.125)], [FL-T200#1, .Real#2680 (1.125)], [FL-T200#2, .Real#1851 (1.125)], [FL-T200#3, .Real#515 (1.125)], [FL-T200#4, .Real#2647 (1.125)], [FL-T200#5, .Real#569 (1.125)], [FL-T200#6, .Real#323 (1.125)], [FL-T200#7, .Real#122 (1.125)], [FL-T200#8, .Real#806 (1.125)], [FL-T200#9, .Real#972 (1.125)], [FL-T200#10, .Real#76 (1.125)], [FL-T200#11, .Real#1782 (1.125)], [FL-T200#12, .Real#591 (1.125)], [FL-T200#13, .Real#1330 (1.125)], [FL-T200#14, .Real#1709 (1.125)], [FL-T200#15, .Real#2410 (1.125)], [FL-T100#0, .Real#2271 (0.5625)], [FL-T100#1, .Real#45 (0.5625)], [FL-T100#2, .Real#1199 (0.5625)], [FL-T100#3, .Real#1948 (0.5625)], [FL-T100#4, .Real#1952 (0.5625)], [FL-T100#5, 

In [19]:
m0_interpretation[fre_2_result]

[[LS#0,
  + ($x, $y) => $result,
  sum ($collection) => $result,
  collect ($collection) => $result,
  FRE.Mass (p) => $result,
  FRE.Mass,
  LS#0.+ ($x, $y) => $result.sum ($collection) => $result.collect ($collection) => $result.FRE.Mass (p) => $result.FRE.Mass.$result#0 ([[FL-T200#0, .Real#2604 (unset)], [FL-T200#1, .Real#1630 (unset)], [FL-T200#2, .Real#2176 (unset)], [FL-T200#3, .Real#3002 (unset)], [FL-T200#4, .Real#1046 (unset)], [FL-T200#5, .Real#2682 (unset)], [FL-T200#6, .Real#20 (unset)], [FL-T200#7, .Real#2187 (unset)], [FL-T200#8, .Real#876 (unset)], [FL-T200#9, .Real#689 (unset)], [FL-T200#10, .Real#2915 (unset)], [FL-T200#11, .Real#1103 (unset)], [FL-T200#12, .Real#594 (unset)], [FL-T200#13, .Real#284 (unset)], [FL-T200#14, .Real#1550 (unset)], [FL-T200#15, .Real#84 (unset)], [FL-T100#0, .Real#68 (unset)], [FL-T100#1, .Real#883 (unset)], [FL-T100#2, .Real#2107 (unset)], [FL-T100#3, .Real#1756 (unset)], [FL-T100#4, .Real#2415 (unset)], [FL-T100#5, .Real#2094 (unset)], [FL

Show all interpretation sequence sets (limited to length of 5).

In [9]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)